In [7]:
from efficientnet_pytorch import EfficientNet

In [2]:
# other imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import transforms as T
import torchvision
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import urllib 

import torchvision.models as models
from PIL import Image

In [8]:
model = EfficientNet.from_pretrained('efficientnet-b0')

Loaded pretrained weights for efficientnet-b0


In [9]:
np.random.seed(13)
# поскольку pytorch опирается на numpy, то верхней строки достаточно
torch.manual_seed(13)

In [10]:
# так требует готовая модель
IMG_DIM = 224
BATCH_SIZE = 128
N_EPOCHS = 3  # больше и так не надо

# так требует готовая модель
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

img_transforms_train = T.Compose([
    T.Resize((IMG_DIM, IMG_DIM)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean_nums, std_nums)
])

img_transforms_test = T.Compose([
    T.Resize((IMG_DIM, IMG_DIM)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean_nums, std_nums)
])

train_dataset = ImageFolder('./images/train/', img_transforms_train)
test_dataset = ImageFolder('./images/test/', img_transforms_test) 

train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=0)

In [11]:
# "заморозить" веса
for param in model.parameters():
    param.requires_grad = False

In [12]:
# тут на кончике огромного efficientnet должна быть маленькая сеть с несколькими FC-слоями,
# но ради эксперимента я решил попрбовать сразу отобразить в два нейрона
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 2)

In [13]:
optimizer = optim.Adam(model.parameters(), 1e-2)
crit = nn.CrossEntropyLoss()

In [18]:
losses = []
def train(model, optimizer, 
          loss_fn, train_loader, n_epochs=5):
    model.train()
    for epoch in range(n_epochs):
        
        for (idx, data) in enumerate(train_loader):
            input, target = data
            
            optimizer.zero_grad()
            output = model(input)
            
            loss = loss_fn(output, target)
            loss.backward()
            
            optimizer.step()
            
            losses.append(loss.item())
            
            if idx % 100 == 0:
                print(f"epoch #{epoch}, iteration #{idx}, loss on each 100 batches = {loss.item()}")

In [19]:
# это оооочень долго на CPU
train(model, optimizer, crit, train_loader)

epoch #0, iteration #0, loss = 0.7287877798080444


KeyboardInterrupt: 